# Solving Krussel-Smith with Deep Equilibrium Nets
*Simon Lebastard, 02/07/2023*

## Krussel-Smith (1998) model
In the KS'98 model, we look a flavour of the stochastic growth model with a continuum of households under partially uninsurable risk.

**Demand side**
$$V(c) = \mathrm{E}_0\Big[\sum_{t=0}^{\infty}{\beta^t U(c_t)}\Big]$$
with
$$U(c) = \lim_{\nu \to \sigma}{\frac{c^{1-\nu} - 1}{1 - \nu}}$$

Agents are each endowed with $\epsilon \tilde{l}$ units of labor per period, where $\epsilon$ is a first-order Markov chain. (In K&S'98, $\epsilon$ can only take two values: 0 and 1, representing unemployed and employed idiosyncratic states, respectively).
As a very large number of agents is consided, and by assumed independence of the idiosyncratic shocks, K&S'98 assume that the total number of employed and unemployed people remain constant over time. That is, there is no aggregate fluctuations of jobs supply.

Idiosyncratic endogenous state: $k$ holding of capital
Idiosyncratic exogenous state: $\epsilon$ at the individual level
$\Gamma$ the joint distribution of idiosyncratic states $(k,\epsilon)$

Given the aggregate states (see $z$ defined below), the consumer's optimization problem is:
$$v(k,\epsilon;\Gamma,z) = \max_{c,k'}{ \Big\{ U(c) + \beta\mathrm{E}\Big[v(k',\epsilon';\Gamma',z') \mid z,\epsilon \Big] \Big\} } $$
under budget constraint, rational expectations wrt law of motion and non-negative capital holding.

The budget constraint writes:
$$c+k' = r(\hat{k},\hat{l},z)k + w(\hat{k},\hat{l},z)\tilde{l}\epsilon + (1-\delta)k$$

**Supply side**
A single-type good is produced using two factors of production: labor $l$ and capital $k$.
The good is produced according to a Cobb-Douglas production function:
$$y = zk^{\alpha}l^{1-\alpha}, \quad \alpha \in [0,1]$$
The TFP is stochastic and coresponds to the source of aggregate risk. In K&S'98, two aggregate states are considered: $(z_b, z_g)$.
Again, $z$ follows a first-order Markov chain.

We assume the production market to be competitive, such that wages $w$ and rental rates $r$, both functions of aggregate states, are respectively determined by:
$$w(\hat{k},\hat{l},z) = (1 - \alpha)z(\frac{\hat{k}}{\hat{l}})^\alpha$$
$$r(\hat{k},\hat{l},z) = \alpha z(\frac{\hat{k}}{\hat{l}})^{\alpha-1}$$

Here we assume that as the population of households is infinitely large, the share of unemployed remains constant. Moreover, here consumers have no disutility from labor, implying that the labor supply at each period is constant at $L_s = N*\mathrm{E}\big[\epsilon\big]$.
We have the market clearing condition for the capital/cons good: $$\int{(c(k,\epsilon;\Gamma,z) + k'(k,\epsilon;\Gamma,z))dF(\Gamma)} = (1-\delta)\int{k dF(\Gamma)} + z\int{k dF(\Gamma)}^{\alpha}L_s^{1-\alpha}$$

For both Markov chains, we assume the economy is already running at the stationary distribution.

**Law of motion**
$$\Gamma' = H(\Gamma,z,z')$$


### Formulating the problem for solving with DEN
Here we will consider two "implicit" policy functions for which we solve:
- Next-period capital $k'(k,\epsilon;\Gamma,z)$
- The Lagrange multiplier on the next-period capital non-negativity constraint: $\mu_k(k,\epsilon;\Gamma,z)$
- The Lagrange multiplier on the positivity of current-period consumption, ie of the residual of the budget constraint: $\mu_c(k,\epsilon;\Gamma,z)$

As in Azinovic et al, we simulate $N$ agents, and index $i \in [1,...,N]$.

#### Idiosyncratic error terms
The Euler equation can be obtained by taking the FOC of the consumer's objective function with respect to next-period capital:
$$1 = \beta(1-\delta)\mathrm{E}\Big[\Big(\frac{c(k,\epsilon;\Gamma,z)}{c(k',\epsilon';\Gamma',z')}\Big)^{\nu} \mid z,\epsilon\Big] - \big(\mu_c(k,\epsilon;\Gamma,z) - \mu_k(k,\epsilon;\Gamma,z) \big)c(k,\epsilon;\Gamma,z)^{\nu}$$

Based on that, we defined the Euler error as:
$$e_{EE}(k,\epsilon,\Gamma,z) \equiv \bigg[\beta(1-\delta)\mathrm{E}\Big[\Big(\frac{c(k,\epsilon;\Gamma,z)}{c(k',\epsilon';\Gamma',z')}\Big)^{\nu} \mid z,\epsilon\Big] - \big(\mu_c(k,\epsilon;\Gamma,z) - \mu_k(k,\epsilon;\Gamma,z) \big)\bigg]^{-\frac{1}{\nu}} - 1$$
and
$$e_{EE,i} \equiv e_{EE}(k_i,\epsilon_i,\Gamma,z)$$

We define the error on the complementary-slackness condition on k as:
$$e_{CS_k}(k,\epsilon,\Gamma,z) \equiv \mu_k(k,\epsilon;\Gamma,z)k'(k,\epsilon;\Gamma,z)$$
and
$$e_{CS_k,i} \equiv e_{CS_k}(k_i,\epsilon_i,\Gamma,z)$$

We define the error on the complementary-slackness condition on c as:
$$e_{CS_c}(k,\epsilon,\Gamma,z) \equiv \mu_c(k,\epsilon;\Gamma,z)c(k,\epsilon;\Gamma,z)$$
and
$$e_{CS_c,i} \equiv e_{CS_c}(k_i,\epsilon_i,\Gamma,z)$$

#### Aggregate error term
At each period, we compute the aggregate $K \equiv \sum_{i=1}^{N}{k_i}$

One way to proceed could be to define an error based on the market clearing condition on capital. Instead, we will enforce consumption to satisfy the market clearing condition at each period. Note that in doing so, we could still need to enforce that consumption is positive. Instead of enforcing a new constraint on a Lagrangian multiplier associated with consumption, I compute the error on MC by constraining consumption to be positive in the error, by using a transformation of the error that satisfies:
$$\lim_{c \downarrow 0}{e_{MC}(c)} = \infty$$
This should prevent consumption from ever being non-positive.

We define the error on the market clearing condition as:
$$e_{MC} \equiv \sum_{i=1}^{N}{\Big[c(k_i,\epsilon_i;\Gamma,z) + k'(k_i,\epsilon_i;\Gamma,z)\Big]} - zK^{\alpha}(Nu)^{1-\alpha} - (1-\delta)K$$

#### Defining the loss function
On a batch $\mathcal{D}_{train}$, the loss function is defined as:
$$l(\theta) \equiv \frac{1}{\mid\mathcal{D}_{train}\mid} \sum_{x \in \mathcal{D}_{train}}{\frac{1}{N-1}\sum_{i=1}^{N}{\Big(e_{EE,i}^2 + e_{CS_k,i}^2 + e_{CS_c,i}^2\Big)} + e_{MC}^2}$$

**Note: alternative implementation**<br>
Instead of having a policy variable for the Lagrange multiplier on the positivity of next-period capital, we could have a transformation function on  next-period capital that ensures it remains non-negative at all times. I will implement this alternative method and compare results.

#### Defining network dimensions
As we make consumption implicit, we have the following dimensions:
- $2N+1$ scalar inputs: $(z, \big\{(k_i,\epsilon_i)\big\}_{i \in [1..N]})$
- $4N$ scalar outputs: ${k_i', c_i, \mu_{k,i}, \mu_{c,i}}_{i \in [1..N]}$

In [1]:
%matplotlib notebook

# Import modules
import os
import re
from datetime import datetime

import tensorflow as tf
tf.config.run_functions_eagerly(True)
import numpy as np

import matplotlib.pyplot as plt
from tqdm import tqdm
from keras.models import Model
from keras.layers import * #Input, Dense, BatchNormalization
from tensorflow import Tensor

# Set the seed for replicable results
seed = 0
np.random.seed(seed)
#tf.set_random_seed(seed)

2023-02-09 12:13:02.609031: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def firm(K,z):
    prod = z*tf.pow(K, α)*tf.pow(L, 1-α)
    r = z*α*tf.pow(K, α-1)*tf.pow(L, 1-α)
    w = z*(1-α)*tf.pow(K, α)*tf.pow(L, -α)
    return prod, r, w

In [3]:
N = num_agents = 100

# Neural network architecture parameters
num_input_nodes = 2*N + 1  # Dimension of extended state space (8 aggregate quantities and 4 distributions)
num_output_nodes = 2*N  # Output dimension (capital holding for each agent. Agent 1 is born without capital (k^1=0))

In [4]:
from tensorflow.random import stateless_binomial

# Shocks structure
Z = tf.constant([0.9, 1.1], dtype=tf.float32)
z_l = Z[0]
z_h = Z[1]
P_z = tf.constant([[0.9, 0.1], [0.1, 0.9]], dtype=tf.float32)
def draw_z(z):
    z_id = int(z==z_h)
    zp_id = stateless_binomial(shape=[1,], counts=1, probs=P_z[z_id,:])
    return Z[zp_id]

E = tf.constant([0, 1], dtype=tf.float32)
eps_l = E[0]
eps_h = E[1]
P_eps = tf.constant([[0.5, 0.5], [0.5, 0.5]], dtype=tf.float32)
def draw_eps(N: int):
    return stateless_binomial(
        shape=[N,],
        seed=[123, 456],
        counts=1,
        probs=0.5,
        output_dtype=tf.float32,
    )

# Other constants
α = tf.constant(0.3, dtype=tf.float32)
β = tf.constant(0.7, dtype=tf.float32)
δ = tf.constant(0.1, dtype=tf.float32)
γ = tf.constant(2.0, dtype=tf.float32)
L = N*tf.reduce_mean(E)

2023-02-09 12:13:05.765667: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Neural Network - Architecture

The following is a specialization class of Keras' Model, with a custom training step with gradient taping

In [5]:
def budget_residual(k: Tensor, c: Tensor, eps:Tensor, kp: Tensor, r:float, w:float):
    return w*eps + (1+r-δ)*k - kp - c

def FB(a:float, b:float):
    return a + b - tf.sqrt(tf.pow(a,2) + tf.pow(b,2))

lr = 0.00001

initializer = tf.keras.initializers.GlorotUniform()
optimizer = tf.keras.optimizers.Adam(
    learning_rate=lr,
)

class DENModel(Model):
    @tf.function
    def forward(self, z:Tensor, k:Tensor, eps:Tensor):
        N = k.shape[0]
        kp = tf.Variable(tf.zeros_like(k))
        mup = tf.Variable(tf.zeros_like(k))
        c = tf.Variable(tf.zeros_like(k))
        lambdap = tf.Variable(tf.zeros_like(k))

        K = tf.math.reduce_sum(k)
        x_aggr = tf.concat([z[None],k],axis=0)
        x_aggr = tf.reshape(x_aggr, shape=[1,-1])
        for agent_id in range(N):
            x_idio = tf.concat([k[agent_id][None], eps[agent_id][None]], axis=0)
            x_idio = tf.reshape(x_idio, shape=[1,-1])
            pol = self(inputs=(x_aggr, x_idio), training=True)
            kp[agent_id].assign(pol[0,0])
            mup[agent_id].assign(pol[0,1])
            c[agent_id].assign(pol[0,2])
            lambdap[agent_id].assign(pol[0,3])
        return kp, mup, c, lambdap

    @tf.function
    def residuals(self, k:Tensor, eps:Tensor, z:float):
        N = k.shape[0]
        K = tf.math.reduce_sum(k)
        Y,r,w = firm(K,z)

        # 1st forward pass
        kp, mup, c, lambdap = self.forward(k, eps, z)
        C = tf.math.reduce_sum(c)
        Kp = tf.math.reduce_sum(kp)
        BUDGET_RES = budget_residual(k, c, eps, kp, r, w)
        CSK_RES = mup*kp
        CSC_RES = c*lambdap

        # For each possible value of next-period exogenous states, compute the next-period policy
        kpp = tf.zeros((N,2,2))
        Kpp = np.zeros((2,2))
        mupp = np.zeros((N,2,2))
        lambdapp = np.zeros((N,2,2))
        cp = np.zeros((N,2,2))
        Cp = np.zeros((2,2))
        ee_comp = np.zeros((N,2,2))
        # BUDGET_RES_COND = np.zeros((N,2,2))
        # CSK_RES_COND = np.zeros((N,2,2))
        # CSC_RES_COND = np.zeros((N,2,2))
        # MC_RES_COND = np.zeros((N,2,2))

        for zp_id, zp in enumerate(Z):
            Yp,rp,wp = firm(Kp,zp)
            for epsp_id, epsp in enumerate(E):
                ypp = self.forward(kp, epsp, zp)
                kpp[:,zp_id,epsp_id] = ypp[0]
                mupp[:,zp_id,epsp_id] = ypp[1]
                cp[:,zp_id,epsp_id] = ypp[2]
                lambdapp[:,zp_id,epsp_id] = ypp[3]
                ee_comp[:,zp_id,epsp_id] = tf.pow(c/cp[zp_id,epsp_id],γ)
                # BUDGET_RES_COND[:,zp_id,epsp_id] = budget_residual(kp, ypp[2], epsp, ypp[0], rp, wp)
                # CSK_RES_COND[:,zp_id,epsp_id] = mupp[:,zp_id,epsp_id]*kpp[:,zp_id,epsp_id]
                # CSC_RES_COND[:,zp_id,epsp_id] = cp[:,zp_id,epsp_id]*lambdapp[:,zp_id,epsp_id]

                Kpp[zp_id,epsp_id] = tf.math.reduce_sum(kpp[:,zp_id,epsp_id])
                Cp[zp_id,epsp_id] = tf.math.reduce_sum(cp[:,zp_id,epsp_id])
                # MC_RES_COND[zp_id,epsp_id] = Cp[zp_id,epsp_id] + Kpp[zp_id,epsp_id] - (1-δ)*Kp - Yp
        
        ee_comp = tf.transpose(ee_comp, [1, 0, 2])
        EE_RES = tf.pow(β*(1-δ)*tf.math.reduce_mean(tf.tensordot(P_z[int(z==z_h)],ee_comp), axis=1) - (lambdap - mup), -1./nu) - 1
        MC_RES = C + Kp - (1-δ)*K - Y
        return BUDGET_RES, CSK_RES, CSC_RES, MC_RES

    @tf.function
    def train_step(self, data, batch_size):
        z, k, eps = data
        ERR = 0
        for per_id in range(batch_size):
            print("Input in train_step: ", z, k ,eps)
            y = self.forward(z, k, eps)
            BUDGET_RES, CSK_RES, CSC_RES, MC_RES = self.residuals(k, eps, z)
            ERR += (1./batch_size)*tf.math.reduce_mean(BUDGET_RES*BUDGET_RES + CSK_RES*CSK_RES + CSC_RES*CSC_RES) + MC_RES*MC_RES
            k = y[0]
            eps = draw_eps(N)
            z = draw_z(z)
        return ERR, z, k, eps

### Compact network approach

In [6]:
n_aggr_repr = 8

## AGGREGATE REPRESENTATION UNITS
# Common network processes distribution-relevant information
x_aggr = Input(shape=(N+1, ), name='Distr-In')
#xn_aggr = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,)(x_aggr)
aggr_1 = Dense(units=4*n_aggr_repr, activation = 'tanh', kernel_initializer=initializer, name='Distr-Dense1')(x_aggr)
aggr_2 = Dense(units=4*n_aggr_repr, activation = 'tanh', kernel_initializer=initializer, name='Distr-Dense2')(aggr_1)
aggr_3 = Dense(units=2*n_aggr_repr, activation = 'tanh', kernel_initializer=initializer, name='Distr-Dense3')(aggr_2)
aggr_4 = Dense(units=n_aggr_repr, activation = 'tanh', kernel_initializer=initializer, name='Distr-Dense4')(aggr_3)

## POLICY UNITS
# Agent-specific policy units
x_idio = Input(shape = (2, ), name='Idio-In')
combined = Concatenate(name='Intermediate_Input')([aggr_4, x_idio])
interp_c_h_1 = Dense(units=32, input_dim=2+n_aggr_repr, activation = 'tanh', kernel_initializer=initializer, name='Policy-Dense1')(combined)
interp_c_h_2 = Dense(units=32, activation = 'tanh', kernel_initializer=initializer, name='Policy-Dense2')(interp_c_h_1)
policy = Dense(units=4, activation = 'tanh', kernel_initializer=initializer, name='Policy-Dense3')(interp_c_h_2)

nn1 = DENModel(inputs = [x_aggr,x_idio], outputs= policy)

/Users/slebst/opt/miniconda3/lib/python3.9/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [7]:
nn1.summary()

Model: "den_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Distr-In (InputLayer)          [(None, 101)]        0           []                               
                                                                                                  
 Distr-Dense1 (Dense)           (None, 32)           3264        ['Distr-In[0][0]']               
                                                                                                  
 Distr-Dense2 (Dense)           (None, 32)           1056        ['Distr-Dense1[0][0]']           
                                                                                                  
 Distr-Dense3 (Dense)           (None, 16)           528         ['Distr-Dense2[0][0]']           
                                                                                          

### Brute-force MC-Distribution-processing neural network

In [49]:
# n_aggr_reprb = 8

## AGGREGATE REPRESENTATION UNITS
## Common network processes distribution-relevant information
# x_aggrb = Input(shape = (None,2))
# xn_aggrb = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True,)(x_aggr)
# aggr_1b = Dense(4*n_aggr_reprb, activation = 'tanh', kernel_initializer=initializer)(xn_aggr)
# aggr_2b = Dense(4*n_aggr_reprb, activation = 'tanh', kernel_initializer=initializer)(aggr_1)
# aggr_3b = Dense(2*n_aggr_reprb, activation = 'tanh', kernel_initializer=initializer)(aggr_2)
# aggr_4b = Dense(n_aggr_reprb, activation = 'tanh', kernel_initializer=initializer)(aggr_3)

## POLICY UNITS
# Agent-specific policy units
# x_idiob = Input(shape = (None,2*N))
# interp_c_h_1b = Dense(32, input_dim=2*N+n_aggr_reprb, activation = 'tanh', kernel_initializer=initializer)(tf.concat(values=(x_idiob,aggr_4b),axis=1))
# interp_c_h_2b = Dense(32, activation = 'tanh', kernel_initializer=initializer)(interp_c_h_1b)
# policyb = Dense(2*N, activation = 'tanh', kernel_initializer=initializer)(interp_c_h_2b)

# nn2 = DENModel(inputs = tf.concat(values=(x_aggrb,x_idiob)), outputs= policyb)

### Model training

In [8]:
def initialize_k(N: int):
    k_ss = 2.
    return tf.Variable(tf.random.uniform(shape=[N,], minval=0.7*k_ss, maxval=1.3*k_ss))

def train(nn: DENModel, N: int=100, n_epochs=1000, batch_size: int=64):
    z = z_h
    eps = draw_eps(N)
    k = initialize_k(N)
    metrics = {'mse': []}

    for epoch in tqdm(range(n_epochs)):
        with tf.GradientTape() as tape:
            ERR, z, k, eps = nn.train_step([z, k, eps], batch_size)
        grads = tape.gradient(ERR, nn.trainable_weights)
        optimizer.apply_gradients(zip(grads, nn.trainable_weights))
        metrics['mse'].append(ERR)

        # Log every 200 batches.
        if epoch % 100 == 0:
            print(
                "Training loss (for one batch) at epoch %d: %.4f"
                % (epoch, float(ERR))
            )
            print("Total # time iterations: %d" % (batch_size*(1+epoch)))

Training time!

In [16]:
train(nn1, N=100)

  0%|          | 0/1000 [00:00<?, ?it/s]

Input in train_step:  Tensor("data:0", shape=(), dtype=float32) Tensor("data_1:0", shape=(100,), dtype=float32) Tensor("data_2:0", shape=(100,), dtype=float32)
x_aggr:  Tensor("concat:0", shape=(101,), dtype=float32)
x_idio:  Tensor("concat_1:0", shape=(2,), dtype=float32)


  0%|          | 0/1000 [00:00<?, ?it/s]


ValueError: in user code:

    File "/var/folders/jz/7fj0cjjx1zbbj_0k1f_617h40000gn/T/ipykernel_15185/3205358969.py", line 91, in train_step  *
        y = self.forward(z, k, eps)
    File "/var/folders/jz/7fj0cjjx1zbbj_0k1f_617h40000gn/T/ipykernel_15185/3205358969.py", line 29, in forward  *
        pol = self(inputs=(x_aggr, x_idio), training=True)
    File "/Users/slebst/opt/miniconda3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/Users/slebst/opt/miniconda3/lib/python3.9/site-packages/keras/engine/input_spec.py", line 250, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encountered when calling layer 'den_model_1' (type DENModel).
    
    Input 0 of layer "Distr-Dense1" is incompatible with the layer: expected min_ndim=2, found ndim=1. Full shape received: (101,)
    
    Call arguments received by layer 'den_model_1' (type DENModel):
   

In [9]:
k = initialize_k(N)
eps = draw_eps(N)

In [10]:
x_aggr = tf.concat([z_h[None],k],axis=0)
x_aggr = tf.reshape(x_aggr, shape=[1,-1])

x_idio = tf.concat([k[0][None], eps[0][None]], axis=0)
x_idio = tf.reshape(x_idio, shape=[1,-1])

nn1(inputs=(x_aggr, x_idio), training=False)

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[ 0.63211966, -0.26554337, -0.14226943,  0.8057285 ]],
      dtype=float32)>

In [11]:
kp, mup, c, lambdap = nn1.forward(z_h,k,eps)